<a href="https://colab.research.google.com/github/sooonsyk/ESAA/blob/main/0516_LGBM_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Lightgbm

* 데이터 '와인품질분류.zip'입니다.

1. 필요한 라이브러리를 가져옵니다.

In [1]:
import pandas as pd
import lightgbm as lgbm

2. 데이터를 불러옵니다.

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
smpl_sub = pd.read_csv('sample_submission.csv')

In [4]:
train.head()

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality
0,1,8.0,0.18,0.37,1.3,0.040,15.0,96.0,0.99120,3.06,0.61,12.1,white,6
1,2,7.5,0.38,0.29,4.9,0.021,38.0,113.0,0.99026,3.08,0.48,13.0,white,7
2,3,6.1,0.27,0.33,2.2,0.021,26.0,117.0,0.98860,3.12,0.30,12.5,white,6
3,4,6.4,0.32,0.50,10.7,0.047,57.0,206.0,0.99680,3.08,0.60,9.4,white,5
4,5,8.4,0.37,0.43,2.3,0.063,12.0,19.0,0.99550,3.17,0.81,11.2,red,7


In [5]:
test.head()

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,1,7.3,0.31,0.25,6.65,0.032,30.0,138.0,0.99244,2.90,0.37,11.1,white
1,2,8.7,0.70,0.24,2.50,0.226,5.0,15.0,0.99910,3.32,0.60,9.0,red
2,3,6.9,0.63,0.01,2.40,0.076,14.0,39.0,0.99522,3.34,0.53,10.8,red
3,4,6.3,0.23,0.50,10.40,0.043,61.0,132.0,0.99542,2.86,0.46,9.1,white
4,5,6.9,0.54,0.04,3.00,0.077,7.0,27.0,0.99870,3.69,0.91,9.4,red


3. 변수를 변환합니다. 

In [11]:
# type에는 white와 red 두 종류가 있습니다.
# 각각 0,1로 변환합니다.

train['type'] = train['type'].apply(lambda x : 0 if x=='white' else 1)
test['type'] = test['type'].apply(lambda x : 0 if x=='white' else 1)

In [12]:
train.head()

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality
0,1,8.0,0.18,0.37,1.3,0.040,15.0,96.0,0.99120,3.06,0.61,12.1,0,6
1,2,7.5,0.38,0.29,4.9,0.021,38.0,113.0,0.99026,3.08,0.48,13.0,0,7
2,3,6.1,0.27,0.33,2.2,0.021,26.0,117.0,0.98860,3.12,0.30,12.5,0,6
3,4,6.4,0.32,0.50,10.7,0.047,57.0,206.0,0.99680,3.08,0.60,9.4,0,5
4,5,8.4,0.37,0.43,2.3,0.063,12.0,19.0,0.99550,3.17,0.81,11.2,1,7


In [13]:
test.head()

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,1,7.3,0.31,0.25,6.65,0.032,30.0,138.0,0.99244,2.90,0.37,11.1,0
1,2,8.7,0.70,0.24,2.50,0.226,5.0,15.0,0.99910,3.32,0.60,9.0,1
2,3,6.9,0.63,0.01,2.40,0.076,14.0,39.0,0.99522,3.34,0.53,10.8,1
3,4,6.3,0.23,0.50,10.40,0.043,61.0,132.0,0.99542,2.86,0.46,9.1,0
4,5,6.9,0.54,0.04,3.00,0.077,7.0,27.0,0.99870,3.69,0.91,9.4,1


In [16]:
# 모델에 입력하기 전 데이터를 정형합니다.
# train_x : train의 'index', 'quality' 피처를 제거하고 저장합니다.
# train_y : train의 'quality'를 저장합니다.
# test_x : test의 'index'를 제거하고 저장합니다.

train_x = train.drop(['id', 'quality'], axis = 1)
train_y = train['quality']
test_x = test.drop('id', axis = 1)


In [17]:
train_x.shape, train_y.shape, test_x.shape

((3231, 12), (3231,), (3231, 12))

In [18]:
train_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,8.0,0.18,0.37,1.3,0.040,15.0,96.0,0.99120,3.06,0.61,12.1,0
1,7.5,0.38,0.29,4.9,0.021,38.0,113.0,0.99026,3.08,0.48,13.0,0
2,6.1,0.27,0.33,2.2,0.021,26.0,117.0,0.98860,3.12,0.30,12.5,0
3,6.4,0.32,0.50,10.7,0.047,57.0,206.0,0.99680,3.08,0.60,9.4,0
4,8.4,0.37,0.43,2.3,0.063,12.0,19.0,0.99550,3.17,0.81,11.2,1


In [19]:
train_y.head()

0    6
1    7
2    6
3    5
4    7
Name: quality, dtype: int64

In [20]:
test_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,7.3,0.31,0.25,6.65,0.032,30.0,138.0,0.99244,2.90,0.37,11.1,0
1,8.7,0.70,0.24,2.50,0.226,5.0,15.0,0.99910,3.32,0.60,9.0,1
2,6.9,0.63,0.01,2.40,0.076,14.0,39.0,0.99522,3.34,0.53,10.8,1
3,6.3,0.23,0.50,10.40,0.043,61.0,132.0,0.99542,2.86,0.46,9.1,0
4,6.9,0.54,0.04,3.00,0.077,7.0,27.0,0.99870,3.69,0.91,9.4,1


4. LightGBM 모델을 생성 및 훈련합니다.

In [23]:
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier()
lgbm_model.fit(train_x,train_y)

LGBMClassifier()

5. 생성한 모델로 예측 데이터를 생성합니다.

In [43]:
preds = lgbm_model.predict(test_x)

array([6, 6, 6, ..., 5, 7, 6])

In [44]:
preds

array([6, 6, 6, ..., 5, 7, 6])

In [26]:
smpl_sub.head()

,id,quality
0,1,-1
1,2,-1
2,3,-1
3,4,-1
4,5,-1


In [27]:
#서브미션 파일의 quality 변수에 예측값을 대입합니다.
smpl_sub['quality'] = preds

In [28]:
smpl_sub.head()

,id,quality
0,1,6
1,2,6
2,3,6
3,4,6
4,5,6


In [ ]:
# csv 파일로 저장합니다.
smpl_sub.to_csv('submission.csv', index=False)

# 2. XGBoost

In [29]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-05-16 09:50:32--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-05-16 09:50:32--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.250.157.102, 142.250.157.101, 142.250.157.100, ...
Connecting to drive.google.com (drive.google.com)|142.250.157.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/00bdpv2274453sb7ej9884hacof8p128/1652694600000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download [following]
--2022-05-16 09:50:32--  https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0r

In [30]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from xgboost import XGBClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [33]:
# 데이터 확인
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [32]:
test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


In [39]:
# 원핫 인코딩 (pd.get_dummies())
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [34]:
# 모델 정의

xgb_model=XGBClassifier()

In [45]:
# 모델 학습

# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수

X=train_one.drop(['quality'],axis=1)
y=train_one['quality']

# fit 메소드를 이용해 모델 학습
xgb_model.fit(X,y)

XGBClassifier(objective='multi:softprob')

In [46]:
# predict 메소드와 test_one 데이터를 이용해 품질 예측
pred=xgb_model.predict(test_one)

In [47]:
pred

array([6, 5, 5, 6, 6, 6, 6, 6, 6, 7, 6, 5, 6, 6, 6, 5, 6, 7, 6, 5, 6, 6,
       5, 7, 6, 5, 5, 5, 5, 5, 7, 6, 5, 5, 5, 6, 5, 5, 6, 5, 6, 5, 6, 6,
       6, 6, 5, 6, 5, 6, 5, 5, 5, 6, 5, 6, 6, 6, 5, 6, 5, 6, 6, 6, 6, 6,
       5, 5, 6, 5, 5, 5, 5, 5, 6, 5, 6, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 5,
       6, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 6, 5, 5, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 5, 7, 7, 6, 6, 5, 6, 6, 5, 5, 6, 6, 6, 7, 5, 5,
       6, 5, 6, 6, 6, 7, 5, 5, 5, 6, 5, 5, 5, 5, 6, 5, 6, 6, 5, 8, 6, 5,
       6, 5, 5, 6, 6, 5, 5, 6, 6, 6, 6, 5, 5, 6, 6, 5, 5, 5, 6, 5, 6, 5,
       6, 5, 6, 6, 6, 5, 6, 7, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6, 6, 7, 5,
       6, 5, 6, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 5, 5, 6, 6, 6, 6, 5, 6, 6,
       6, 6, 6, 6, 7, 6, 6, 5, 6, 5, 6, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 7, 6, 6, 6, 5, 7, 5, 5, 7, 7, 6, 6, 6, 7, 7, 6, 6, 5, 5, 6,
       5, 5, 6, 6, 6, 6, 6, 5, 5, 5, 6, 5, 7, 6, 5, 5, 6, 6, 6, 6, 6, 6,
       6, 6, 5, 6, 5, 5, 5, 6, 5, 6, 8, 6, 5, 7, 6,

In [ ]:
# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기


In [ ]:
# 정답파일 내보내기
submission.to_csv('xgb_pred.csv',index = False)